In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector, LSTM, concatenate , Input, Reshape
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
from keras import backend as K 
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.models import Model
K.set_image_dim_ordering('th')

Using TensorFlow backend.
/Users/ewallner/.brew/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
max_caption_len = 3
vocab_size = 4

In [4]:
img_path = 'screenshot.jpg'
img = image.load_img(img_path, target_size=(299, 299))
img = image.img_to_array(img)
images = []
for i in range(3):
    images.append(img)
images = np.expand_dims(images, axis=0)
images = preprocess_input(images)

In [9]:
html_input = np.array([[[0,0,0,1],
                        [0,0,0,0], 
                        [0,0,0,0],
                        [0,0,0,0]], 
                       [[0,0,0,1],
                        [0,0,1,0], 
                        [0,0,0,0],
                        [0,0,0,0]], 
                       [[0,0,0,1],
                        [0,0,1,0], 
                        [0,1,0,0],
                        [0,0,0,0]]])
next_words = np.array([[[0,0,1,0], [0,1,0,0], [1,0,0,0]]])
print(html_input.shape)

(3, 4, 4)


In [6]:
IR2 = InceptionResNetV2(weights=None, include_top=True)
ir2_out = Dense(1024, activation='relu')(IR2.output)
ir2_out = RepeatVector(max_caption_len)(ir2_out)

In [7]:
language_model = Input(shape=(max_caption_len, vocab_size))
language_model = LSTM(128, return_sequences=True)(language_model)

In [8]:
print(ir2_out.shape, language_model.shape)
decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=False)(decoder)
decoder = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[IR2.input, language_model], outputs=decoder)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

(?, 3, 1024) (?, ?, 128)


/Users/ewallner/Library/Python/3.6/lib/python/site-packages/keras/engine/topology.py:1574: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "model_1" was not an Input tensor, it was generated by layer lstm_1.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: lstm_1/transpose_1:0
  str(x.name))


TypeError: Input layers to a `Model` must be `InputLayer` objects. Received inputs: [<tf.Tensor 'input_1:0' shape=(?, 3, ?, ?) dtype=float32>, <tf.Tensor 'lstm_1/transpose_1:0' shape=(?, ?, 128) dtype=float32>]. Input 1 (0-based) originates from layer type `LSTM`.

In [ ]:
model.fit([img, html_input], next_words, batch_size=1, nb_epoch=5)